<br>**NAME : HARSH HIMMATBHAI DHAMELEEYA**
<br>**STUDENT NUMBER : 8926600**
<br>**LAB 10 : Fine-Tune VGG16 for Bird Classification**

**TASK 1. OBTAIN DATA**

Importing libraries 

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
base_dir = "D:/assignments/CSCN8010-lab-submissions/students_submissions/8926600/Lab-10_files" 
train_dir = os.path.join(base_dir, 'train')

**TASK 2. Use data-augmentation to increase the number of training images. You are encouraged to try out various augmentation methods supported by Keras. The rule of thumb is that as long as an augmentation produces realistic images - it is probably a good idea to add it.**

Data Augmentation Setup

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

Load the Data

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224), 
    batch_size=32,
    class_mode='categorical'
)  

Found 84635 images belonging to 525 classes.


 Load and Modify VGG16

In [5]:
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the convolutional base
vgg_base.trainable = False

# Create a new model on top
model = models.Sequential()
model.add(vgg_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))  

Compile and Train the Model

In [6]:
validate_path = "D:/assignments/CSCN8010-lab-submissions/students_submissions/8926600/Lab-10_files/valid/"
validation_generator = train_datagen.flow_from_directory(
    validate_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
)

from tensorflow.keras.applications import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

from tensorflow.keras import layers
x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(3, activation='softmax')(x)  

model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())

Found 2625 images belonging to 525 classes.


Fine-Tuning

In [7]:
model.add(layers.Dense(525, activation='softmax'))
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


# Validation data generator
val_datagen = ImageDataGenerator(rescale=1./255)
validation_dir = os.path.join(base_dir, 'valid') 
validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='categorical'
)


Found 2625 images belonging to 525 classes.


 Save and Evaluate the Model

In [10]:
# Fit the model
history = model.fit(
    train_generator,  
    steps_per_epoch=100,
    epochs=3,  # We can increase this number to get higher accuracy
    validation_data=validation_generator,
    validation_steps=50
)

# Test data generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_dir = "D:/assignments/CSCN8010-lab-submissions/students_submissions/8926600/Lab-10_files/test"
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, steps=50)

Epoch 1/3
100/100 [==============================] - 556s 6s/step - loss: 10.3665 - accuracy: 0.0137 - val_loss: 10.5385 - val_accuracy: 0.0170
Epoch 2/3
100/100 [==============================] - 549s 5s/step - loss: 9.6035 - accuracy: 0.0219 - val_loss: 8.9303 - val_accuracy: 0.0360
Epoch 3/3
100/100 [==============================] - 549s 5s/step - loss: 9.1668 - accuracy: 0.0338 - val_loss: 7.4223 - val_accuracy: 0.0950
Found 2625 images belonging to 525 classes.
50/50 [==============================] - 202s 4s/step - loss: 7.7723 - accuracy: 0.0681
